# ExTensor

This notebook reproduces the salient characteristics of the [ExTensor](https://dl.acm.org/doi/10.1145/3352460.3358275) accelerator.

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src import utils

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below.

**Warning:** Large tensors will overwhelm the video generation. Either:
1. Use small tensors; as a rule of thumb, fewer than 60 computes (e.g., multiplications) should be required.
2. Do not generate a video; remove the `spacetime` specification from the `mapping` before compiling.

In [ ]:
K = 8
M = 4
N = 4

density = [1, 0.75]
seed = 0

K0 = 2
K1 = 4
M0 = 2
M1 = 4
N0 = 2
N1 = 4

A_KM = Tensor.fromRandom(rank_ids=["K", "M"], shape=[K, M], seed=seed, density=density, name="A")
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=density, name="B")

## Compile and Run

Below is the TeAAL specification for ExTensor. To simulate the accelerator:
1. Compile it to HiFiber by running the cell, inserting a new cell
2. Run the new cell, which will
    - Execute the kernel; multiplying the above defined matrices
    - Generate visualizations of the actions of the kernel

#### Notes
- The dataflow described in the ExTensor paper differs from the dataflow implemented in its simulator and used to collect its results. The following specification describes the Einsum and mapping for the actual simulator.
- Small tensors are required for video generation. If you are using large tensors, remove the spacetime specification to generate a kernel that does not produce videos. Outputs can still be checked below.
- Partition shapes are decreased accordingly above for visualization purposes. The real ExTensor uses problem-specific values for `K0`, `K1`, `M0`, `M1`, `N0`, and `N1`.

In [ ]:
yaml = """
einsum:
  declaration:
    A: [K, M]
    B: [K, N]
    Z: [M, N]
  expressions:
    - Z[m,n] = A[k,m] * B[k,n]
mapping:
  rank-order:
    A: [K, M]
    B: [K, N]
    Z: [M, N]
  partitioning:
    Z:
      K:
        - uniform_shape(K1)
        - uniform_shape(K0)
      M:
        - uniform_shape(M1)
        - uniform_shape(M0)
      N:
        - uniform_shape(N1)
        - uniform_shape(N0)
  loop-order:
    Z: [N2, K2, M2, M1, N1, K1, M0, N0, K0]
  spacetime:
    Z:
      space: [K1]
      time: [N2, K2, M2, M1, N1, M0, N0, K0]
"""

utils.compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell).

In [ ]:
utils.check_matmul(A_KM, B_KN, Z_MN)